In [1]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ff_energy.ffe.utils import read_from_pickle
import jax.numpy as jnp
from ff_energy.ffe.plot import plot_energy_MSE, plot_ff_fit
from ff_energy.ffe.structure import atom_key_pairs
from ff_energy.ffe.potential import LJ, akp_indx
from ff_energy.ffe.ff import FF
from ff_energy.ffe.ff_fit import LJ_bound, load_ff, fit_func, fit_repeat
from ff_energy.ffe.data import pairs_data
from ff_energy.ffe.utils import pickle_output, read_from_pickle, str2int




In [17]:
NDE = 10
NLJ = 10
sig_bound = (0.05, 2.5)
ep_bound = (0.001, 0.5)
LJ_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound))
DE_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound), 
            (1, 8), (6, 20))

In [18]:
#  example
# ffpkl = "ECOL_pbe0dz_FOE.ff.pkl"

In [19]:
# Parameters
ffpkl = "kmdcm_pbe0dz_FOE.ff.pkl"


#  LJ fit


In [20]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))
_c

FF: LJ water_cluster ELEC harmonic first_order_energy (jax_coloumb: True)

In [21]:
# _c.targets

In [22]:
fit_repeat(_c, 
           NLJ, 
           f"{ffpkl}_LJ",
           bounds = LJ_bound,
           loss="jax",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:174: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [23]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
6,18.095881,676,1169,0,True,Optimization terminated successfully.,"[0.7079326727067781, 2.495026644804957, 0.0010...","([[0.7079326727067781, 2.495026644804957, 0.00..."
5,18.231602,787,1343,0,True,Optimization terminated successfully.,"[0.547484215886817, 2.5, 0.003561161564913339,...","([[0.547484215886817, 2.5, 0.00356116156491333..."
7,18.334091,752,1288,0,True,Optimization terminated successfully.,"[0.7515402870377843, 2.3568970546176278, 0.001...","([[0.7515402870377843, 2.3568970546176278, 0.0..."
4,19.023872,465,803,0,True,Optimization terminated successfully.,"[0.1456369825487122, 2.4999999968086404, 0.101...","([[0.1456369825487122, 2.4999999968086404, 0.1..."
3,19.132883,714,1211,0,True,Optimization terminated successfully.,"[0.11002068210183372, 2.499999905923318, 0.129...","([[0.11002068210183372, 2.499999905923318, 0.1..."
1,20.915262,488,827,0,True,Optimization terminated successfully.,"[0.05000000000000003, 2.374431012915543, 0.5, ...","([[0.05000000000000003, 2.374431012915543, 0.5..."
0,134.150162,290,529,0,True,Optimization terminated successfully.,"[1.7589109905785407, 1.3214693161425324, 0.001...","([[1.7589109905785407, 1.3214693161425324, 0.0..."
9,162.699814,295,518,0,True,Optimization terminated successfully.,"[1.8071108696025608, 1.0989013642382768, 0.001...","([[1.8071108696025608, 1.0989013642382768, 0.0..."
2,8074.825684,51,152,0,True,Optimization terminated successfully.,"[0.08850058592580834, 0.05, 0.0914911889625608...","([[0.08850058592580834, 0.05, 0.09149118896256..."
8,8074.825684,51,160,0,True,Optimization terminated successfully.,"[0.054369236451822844, 0.0743765904674521, 0.3...","([[0.054369236451822844, 0.0743765904674521, 0..."


#  DE fit


In [24]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))

In [25]:
fit_repeat(_c, 
           NDE, 
           "test",
           bounds = DE_bound,
           loss="jax_de",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:174: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [26]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
2,11.436456,813,1347,0,True,Optimization terminated successfully.,"[0.3543580699154079, 2.5, 0.4031838333027614, ...","([[0.3543580699154079, 2.5, 0.4031838333027614..."
8,11.558370,1218,1971,0,True,Optimization terminated successfully.,"[0.5520535394959611, 2.2575508596793585, 0.156...","([[0.5520535394959611, 2.2575508596793585, 0.1..."
5,11.834990,495,828,0,True,Optimization terminated successfully.,"[0.5411929779291077, 2.093205985616014, 0.1519...","([[0.5411929779291077, 2.093205985616014, 0.15..."
3,12.023440,633,1082,0,True,Optimization terminated successfully.,"[0.38331287403637126, 2.1661217719749857, 0.42...","([[0.38331287403637126, 2.1661217719749857, 0...."
6,12.270730,254,483,0,True,Optimization terminated successfully.,"[0.3828963049376085, 2.1284276146016583, 0.477...","([[0.3828963049376085, 2.1284276146016583, 0.4..."
0,17.292423,1441,2289,0,True,Optimization terminated successfully.,"[2.2957661865281085, 1.5979128395758158, 0.001...","([[2.2957661865281085, 1.5979128395758158, 0.0..."
1,17.559095,1052,1704,0,True,Optimization terminated successfully.,"[2.4900669030088336, 1.5681651408555544, 0.001...","([[2.4900669030088336, 1.5681651408555544, 0.0..."
7,19.102196,1273,2038,0,True,Optimization terminated successfully.,"[1.9645777794789478, 1.5296375356620242, 0.001...","([[1.9645777794789478, 1.5296375356620242, 0.0..."
4,25.222342,1220,1988,0,True,Optimization terminated successfully.,"[1.9060071948016988, 1.8333672852322513, 0.008...","([[1.9060071948016988, 1.8333672852322513, 0.0..."
9,8074.825684,44,198,0,True,Optimization terminated successfully.,"[0.05, 0.2187029725006433, 0.15184742911742766...","([[0.05, 0.2187029725006433, 0.151847429117427..."
